# L'exécution de cette cellule démarre l'application Spark

In [1]:
# test

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1716558371122_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Installation des packages 

In [2]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Définition des PATH  chargement les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [3]:
PATH = 's3://ngor-yoff'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://ngor-yoff
PATH_Data:   s3://ngor-yoff/Test
PATH_Result: s3://ngor-yoff/Results

### Traitement des données

#### Chargement des données

In [4]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://ngor-yoff/Te...|2024-05-11 06:27:40|  7353|[FF D8 FF E0 00 1...|
|s3://ngor-yoff/Te...|2024-05-11 06:28:35|  7350|[FF D8 FF E0 00 1...|
|s3://ngor-yoff/Te...|2024-05-11 06:28:19|  7349|[FF D8 FF E0 00 1...|
|s3://ngor-yoff/Te...|2024-05-11 06:28:17|  7348|[FF D8 FF E0 00 1...|
|s3://ngor-yoff/Te...|2024-05-11 06:28:23|  7328|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------+----------+
|path                                        |label     |
+--------------------------------------------+----------+
|s3://ngor-yoff/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://ngor-yoff/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://ngor-yoff/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://ngor-yoff/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://ngor-yoff/Test/Watermelon/r_95_100.jpg |Watermelon|
+--------------------------------------------+----------+
only showing top 5 rows

None

# Chargement du Modèle MobileNetV2

In [8]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 1s 0us/step

In [9]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [12]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

#### Repartitionnement et Sélection des Colonnes
- distribuetion des données dans 24 partitions ce qui peut être utile pour équilibrer la charge de travail sur les nœuds du cluster et optimiser le parallélisme.
- images. col sont  utilisés pour référencer les colonnes du DataFrame de manière explicite.
-featurize_udf est une fonction utilisateur définie pour Spark (UDF) qui applique la fonction de featurization sur la colonne content.

In [14]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://ngor-yoff/Results

In [16]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
features_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[path: string, label: string, features: array<float>]

#### Lecture du Fichier Parquet et Conversion en Pandas

In [18]:
df_pca.write.mode("overwrite").parquet(PATH_PCA)df = spark.read.format("parquet").load(PATH_Result).toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                             path  ...                                           features
0    s3://ngor-yoff/Test/Watermelon/r_102_100.jpg  ...  [0.014875885099172592, 0.27422279119491577, 0....
1    s3://ngor-yoff/Test/Watermelon/r_100_100.jpg  ...  [0.07104229182004929, 0.3109641373157501, 0.0,...
2   s3://ngor-yoff/Test/Pineapple Mini/40_100.jpg  ...  [0.0, 4.172328948974609, 0.0, 0.0, 0.0, 0.0, 0...
3  s3://ngor-yoff/Test/Pineapple Mini/258_100.jpg  ...  [0.0, 4.659770965576172, 0.0, 0.0, 0.0, 0.0, 0...
4  s3://ngor-yoff/Test/Pineapple Mini/301_100.jpg  ...  [0.0, 4.407875061035156, 0.0, 0.0, 0.0, 0.0172...

[5 rows x 3 columns]

In [20]:
len(df.loc[0,'features'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1280

# PCA 

### Importation des Bibliothèques Nécessaires

In [22]:
from pyspark.ml.functions import array_to_vector 
from pyspark.ml.feature import PCA, StandardScaler
features_df = features_df.select("path","label",array_to_vector("features").alias("features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Définir et Appliquer StandardScaler
La normalisation des données est une étape clé dans le prétraitement des données pour garantir que chaque caractéristique contribue de manière équitable à l'analyse ou à l'apprentissage automatique

#### Ajuster le Scaler aux Données :

In [27]:
scaler = StandardScaler(inputCol= "features", outputCol = "standard_scale_features",withMean = True , withStd= True).fit(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Transformer les Données 

In [28]:
df_scaled = scaler.transform(features_df)
df_scaled.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+-----------------------+
|                path|         label|            features|standard_scale_features|
+--------------------+--------------+--------------------+-----------------------+
|s3://ngor-yoff/Te...|    Watermelon|[0.85376590490341...|   [0.91871372453972...|
|s3://ngor-yoff/Te...|Pineapple Mini|[0.0,5.1022534370...|   [-0.7469990057685...|
+--------------------+--------------+--------------------+-----------------------+
only showing top 2 rows

#### Application de PCA
PCA est une méthode de réduction de dimensionnalité qui projette les données sur un espace de dimension inférieure tout en conservant autant que possible la variance des données d'origine.  
on va  appliquer PCA sur les données normalisées 

cette methode applique la transformation PCA aux données et ajoute une nouvelle colonne avec les caracteristiques réduites 

In [31]:
pca = PCA(k=10,inputCol = "standard_scale_features",outputCol= "pca_features").fit(df_scaled)
df_pca = pca.transform(df_scaled)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Sauvegarde des resultats

In [32]:
df_pca.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_pca.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+-----------------------+--------------------+
|                path|     label|            features|standard_scale_features|        pca_features|
+--------------------+----------+--------------------+-----------------------+--------------------+
|s3://ngor-yoff/Te...|Watermelon|[0.18860274553298...|   [-0.3790316362576...|[-7.8621198274872...|
|s3://ngor-yoff/Te...|Watermelon|[0.16857005655765...|   [-0.4181157749622...|[-1.2965653355910...|
|s3://ngor-yoff/Te...|Watermelon|[0.0,0.2240793704...|   [-0.7469990057685...|[-4.7746594580498...|
|s3://ngor-yoff/Te...|Watermelon|[0.11681207269430...|   [-0.5190965380978...|[-3.3856890337140...|
|s3://ngor-yoff/Te...| Raspberry|[0.01002115849405...|   [-0.7274475442026...|[-2.9605153352219...|
+--------------------+----------+--------------------+-----------------------+--------------------+
only showing top 5 rows